https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb#scrollTo=GWiVUF0jIrIv

https://huggingface.co/transformers/custom_datasets.html#token-classification-with-w-nut-emerging-entities

https://huggingface.co/blog/ray-tune


# Installing and importing libraries

In [ ]:
# !sudo apt install git-lfs
# !git config --global user.email aditeya.baral@gmail.com
# !git config --global user.name Aditeya
HF_HUB_API_TOKEN = "api_vOKpxbhcEnOqvVNYfPxOSzFoMsFNxOqttx"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!pip install transformers datasets seqeval ray

In [ ]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import json
import math
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset, load_metric, DatasetDict
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, RobertaTokenizerFast

# Loading Dataset

In [ ]:
df = pd.read_json('/content/drive/MyDrive/Maple/ppl-grammar-dataset.json')
print(df.shape)
df.head()

(54629, 5)


,poem,haiku,indices,ppl-gpt2,grammar-check
0,Did the CIA tell the FBI that it knows the wor...,cia fbi the biggest weapon,"[2, 5, 9, 24, 25]",2447.346480,False
1,Did the CIA tell the FBI that it knows the wor...,cia fbi the biggest weapon,"[2, 5, 9, 24, 25]",2447.346480,False
2,"Dark clouds gathered overhead,\nExpelling bull...",clouds overhead bullets of the valley,"[1, 3, 5, 6, 10, 11]",3639.095887,False
3,A vigilante lacking of heroic qualities that\n...,lacking qualities that damn criminals,"[2, 5, 6, 11, 12]",8305.684147,False
4,"(A Diamante Poem)\nBrain\nHeavenly, hellish\nF...",diamante poem the sybaritic pathetic,"[1, 2, 10, 18, 19]",1982.106818,False


In [ ]:
df["ppl-gpt2"].describe()

count     54629.000000
mean       6511.533484
std       13967.007052
min           4.944884
25%        1245.401788
50%        2755.530497
75%        6398.639543
max      560770.619276
Name: ppl-gpt2, dtype: float64

In [ ]:
df["grammar-check"].describe()

count     54629
unique        2
top       False
freq      54557
Name: grammar-check, dtype: object

There are some hindi sentences here - clean them in the preprocessing notebook

In [ ]:
filtered_df = df[df["ppl-gpt2"] <= 1500]
filtered_df.reset_index(drop=True, inplace=True)
print(filtered_df.shape)
filtered_df.head()

(16576, 5)


,poem,haiku,indices,ppl-gpt2,grammar-check
0,"You sit left to me,\nYou don't wanna see,\nWha...",name extentions problem was about a tightrope,"[38, 53, 56, 60, 75, 89, 90]",850.798879,False
1,"Life,\nFrom ABC to XYZ;\nLearning all the time...",abc xyz all perfect laws of all rules,"[2, 4, 6, 11, 14, 15, 18, 20]",1279.585696,False
2,"Life,\nFrom ABC to XYZ;\nLearning all the time...",perfect laws all rules like the beginning,"[11, 14, 18, 20, 24, 25, 26]",1097.296416,False
3,"Alfons Schuhbeck top chef.\nBavaria, Germany i...",chef chefs a wonderful cookbook,"[3, 11, 14, 15, 16]",974.032754,False
4,"Actress Angela Oberer passing a moment, Angela...",moment eyes a long time,"[5, 27, 34, 35, 36]",445.035082,False


# Preprocessing Dataset

In [ ]:
filtered_df["tokens"] = filtered_df["poem"].apply(lambda x: word_tokenize(x))
ner_tags = list()
for i in range(filtered_df.shape[0]):
  indices = filtered_df["indices"][i]
  length = len(filtered_df["tokens"][i])
  ner_tag = ['O' for _ in range(length)]
  for idx in indices:
    ner_tag[idx] = 'W'
  ner_tags.append(ner_tag)
filtered_df["ner_tags"] = ner_tags

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
filtered_df.head()

,poem,haiku,indices,ppl-gpt2,grammar-check,tokens,ner_tags
0,"You sit left to me,\nYou don't wanna see,\nWha...",name extentions problem was about a tightrope,"[38, 53, 56, 60, 75, 89, 90]",850.798879,False,"[You, sit, left, to, me, ,, You, do, n't, wan,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"Life,\nFrom ABC to XYZ;\nLearning all the time...",abc xyz all perfect laws of all rules,"[2, 4, 6, 11, 14, 15, 18, 20]",1279.585696,False,"[Life, ,, From, ABC, to, XYZ, ;, Learning, all...","[O, O, W, O, W, O, W, O, O, O, O, W, O, O, W, ..."
2,"Life,\nFrom ABC to XYZ;\nLearning all the time...",perfect laws all rules like the beginning,"[11, 14, 18, 20, 24, 25, 26]",1097.296416,False,"[Life, ,, From, ABC, to, XYZ, ;, Learning, all...","[O, O, O, O, O, O, O, O, O, O, O, W, O, O, W, ..."
3,"Alfons Schuhbeck top chef.\nBavaria, Germany i...",chef chefs a wonderful cookbook,"[3, 11, 14, 15, 16]",974.032754,False,"[Alfons, Schuhbeck, top, chef, ., Bavaria, ,, ...","[O, O, O, W, O, O, O, O, O, O, O, W, O, O, W, ..."
4,"Actress Angela Oberer passing a moment, Angela...",moment eyes a long time,"[5, 27, 34, 35, 36]",445.035082,False,"[Actress, Angela, Oberer, passing, a, moment, ...","[O, O, O, O, O, W, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
train_df, test_df = train_test_split(filtered_df, test_size=0.05)
print(train_df.shape, test_df.shape)

(15747, 7) (829, 7)


In [ ]:
tokens = filtered_df["tokens"]
tags = filtered_df["ner_tags"]

In [ ]:
unique_tags = ["O", "W"]
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Initialising GPT-2 for Perplexity Loss

In [ ]:
gpt2_model_name = "gpt2"
gpt2_model = GPT2LMHeadModel.from_pretrained(gpt2_model_name)
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained(gpt2_model_name)

def perplexityGPT2(sentences):
  ppl = list()
  total_length = len(sentences)
  for index, sent in enumerate(sentences):
    tokenize_input = gpt2_tokenizer.encode(sent)
    tensor_input = torch.tensor([tokenize_input])
    loss = gpt2_model(tensor_input, labels=tensor_input)[0]
    ppl.append(math.exp(loss))
  return ppl

# Setting Model Parameters

In [ ]:
model_name = "bert-large-cased"

In [ ]:
if "roberta" in model_name:
  tokenizer = RobertaTokenizerFast.from_pretrained(model_name, add_prefix_space=True)
else:
  tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
train_texts = list(train_df["tokens"].values)
val_texts = list(test_df["tokens"].values)

train_tags = list(train_df["ner_tags"].values)
val_tags = list(test_df["ner_tags"].values)

In [ ]:
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [ ]:
def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        try:
          doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
          encoded_labels.append(doc_enc_labels.tolist())
        except:
          pass

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [ ]:
class MapleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

offset_mapping_train = train_encodings.pop("offset_mapping")
offset_mapping_val = val_encodings.pop("offset_mapping")
train_dataset = MapleDataset(train_encodings, train_labels)
val_dataset = MapleDataset(val_encodings, val_labels)

In [ ]:
'''def model_init():
  return AutoModelForTokenClassification.from_pretrained(model_name, num_labels=2, return_dict=True)'''

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=2)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be ab

In [ ]:
class PerplexityTrainer(Trainer):
  def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
  def compute_loss(self, model, inputs, return_outputs=False):
    batch_size = len(inputs["input_ids"])
    poems = list()
    for i in range(batch_size):
      text = tokenizer.decode(inputs["input_ids"][i])
      tokenized_text = tokenizer.tokenize(text)
      labels = inputs["labels"][i].tolist()
      words = list()
      for idx, value in enumerate(labels):
        if value == 1:
          words.append(tokenized_text[idx])
      poem = ' '.join(words)
      poems.append(poem)
    perplexity_scores = perplexityGPT2(poems)
    perplexity_loss = sum(perplexity_scores)
    loss = torch.tensor(perplexity_loss, requires_grad=True)
    return (loss, poems) if return_outputs else loss

In [ ]:
args = TrainingArguments(
    f"modelfolder",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=2e-8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

In [ ]:
trainer = PerplexityTrainer(
    # model_init=model_init,                        
    model=model,
    args=args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset             
)

In [ ]:
'''trainer.hyperparameter_search(
    direction="maximize", 
    backend="ray", 
    n_trials=10
)'''

'trainer.hyperparameter_search(\n    direction="maximize", \n    backend="ray", \n    n_trials=10\n)'

# Training and Evaluating Model

In [ ]:
trainer.train()

ValueError: ignored

In [ ]:
trainer.evaluate()

# Saving Model

In [ ]:
model.push_to_hub(f"maple-{model_name.lower()}", use_auth_token=HF_HUB_API_TOKEN)
# tokenizer.push_to_hub(f"maple-{model_name.lower()}", use_auth_token=HF_HUB_API_TOKEN)

In [ ]:
model.save_pretrained(f"/content/drive/MyDrive/Maple/maple-{model_name.lower()}")
trainer.save_model(f"/content/drive/MyDrive/Maple/maple-{model_name.lower()}")